In [36]:
# using POMDPs
# using POMDPModels
# using POMDPSimulators
# using SARSOP
# using QMDP
# using BasicPOMCP
# using DMUStudent

using QuickPOMDPs
using POMDPModelTools

using Random
rng = MersenneTwister(1234);

## 2D Discrete Representation Experiment

Frame the camera search problem in a 2D Discrete world to work out the kinks of a Julia POMDPs implementation.

In [2]:
# utility functions
function zoom_to_FOV(zoom)
    """This function is currently simple, but later may contain
    more complex calculations linking the zoom independent 
    (controlled) variable to other FOV paramters.
    """
    calc_focal_pt = zoom # just a 1-1 for now in this discrete world
    calc_FOV_width = 4 - zoom # an inverse relationship with hardcoded FOV max/min here
    return [calc_focal_pt, calc_FOV_width]
end

function clamp(n, smallest, largest)
    """Useful clamping function"""
    return max(smallest, min(n, largest))
end

clamp (generic function with 1 method)

In [3]:
# define a struct for the relevant problem state information
struct GW_state
    # camera state information
    cam_theta
    zoom
    focal_pt
    FOV_width
    
    # target state information
    target_theta
    target_r
    theta_dot
    r_dot
end

In [41]:
function initialstate(rng)
    # generate a state
    i_cam_theta = rand(rng,(1, 36))
    i_zoom = rand(rng,(1, 3))
    i_focal_pt, i_FOV_width = zoom_to_FOV(i_zoom)
    
    # target state information
    i_target_theta = rand(rng,(1, 36))
    i_target_r = rand(rng,(1, 3))
    i_theta_dot = 0
    i_r_dot = 0
    
    init_state = GW_state(
        i_cam_theta, i_zoom, i_focal_pt, i_FOV_width,
        i_target_theta, i_target_r, i_theta_dot, i_r_dot,
    )
    return init_state 
end

initialstate (generic function with 1 method)

In [5]:
function gen(s, a, rng)
    # generate a named tuple with :sp pointing to the new state
    
    # camera transitions
    #############################################################
    # need to be a function of zoom to account for speed decrease
    # at deep zoom. write up include pix / meter / second at zooms
    # aka the below says that the camera can pan faster when it
    # is zoomed out, which matches real physics
    if a == :left
        if s.zoom <= 2
            sp_cam_theta += 2
        elseif s.zoom > 2
            sp_cam_theta += 1
        end
    elseif a == :stay
        # do nothing
    elseif a == :right
        if s.zoom <= 2
            sp_cam_theta -= 2
        elseif s.zoom > 2
            sp_cam_theta -= 1
        end
    else
        # do nothing
    end

    # handle wrap around
    sp_cam_theta = sp_cam_theta % 36

    # update zoom
    if a == :zoom_in
        sp_zoom += 1
    elseif a == :stay_zoom
        # do nothing
    elseif a == :zoom_out
        sp_zoom -= 1
    end
    sp_zoom = clamp(sp_zoom, 1, 3)

    # update dependent state variables
    sp_focal_pt, sp_FOV_width = zoom_to_FOV(sp_zoom)
    
    # target transitions
    #############################################################   
    
    moving_target = false
    if moving_target
        # if want a moving target then add noise to
        # velocity to create more realistic motion
        
        # center at 0 mean
        sp_theta_dot += rand(rng) - 0.5 
        sp_r_dot += rand(rng) - 0.5 
        
        # clip to prevent crazy vels
        sp_theta_dot = clamp(sp_theta_dot, -1.9, 1.9) 
        sp_r_dot = clamp(sp_r_dot, -1, 1) 
        
    else
        # stationary target
        sp_theta_dot = 0
        sp_r_dot = 0
    end

    # update target state
    sp_target_theta += int(sp_theta_dot)
    sp_target_theta = sp_target_theta % 36
    sp_target_r += int(sp_r_dot)
    sp_target_r = clamp(sp_target_r, 1,3)

    new_state = GW_state(
        sp_cam_theta, sp_zoom, sp_focal_pt, sp_FOV_width,
        sp_target_theta, sp_target_r, sp_theta_dot, sp_r_dot,
    )
    return (sp=new_state,)
end

gen (generic function with 1 method)

In [6]:
function observations(s, a, sp)
    # should return a POMDPModelTools.BoolDistribution

    """Returns a boolean observation depending on observation
    model probabilities that are state dependent.
    """
    # if in FOV 
    # TODO: add handling wrap around to this check
    if abs(s.cam_theta - s.target_theta) <= ceil(s.FOV_width/2)
        # observation based on range
        if s.target_r == s.focal_pt 
            # in FOV and in focal range
            # 90% True Positive detection in this case
            return POMDPModelTools.BoolDistribution(0.9)
        else 
            # in FOV and not at correct focal range
            # 40% True Positive detection in this case
            return POMDPModelTools.BoolDistribution(0.4)
        end
    else
        # if not in FOV
        # 5% False Positive rate in this case
        return POMDPModelTools.BoolDistribution(0.05)
    end
end

observations (generic function with 1 method)

In [11]:
function actions()
    # should return a vector of actions available at the state
    A = [:left, :stay, :right, :zoom_in, :stay_zoom, :zoom_out, :track]
    return A
end

actions (generic function with 2 methods)

In [8]:
function reward(s, a, sp)
    """Return a positive reward if the action is to "Track"
    and a target is in the FOV.
    Track = begin a track since a target is confidently detected.
    Return a negative reward if the action is to "Track" but
    the target is not in the FOV.
    """

    if a == :track
        # if in FOV 
        # TODO: add handling wrap around to this check
        if abs(s.cam_theta - s.target_theta) <= ceil(s.FOV_width/2)
            return 10.0 # and terminal state
        else
            return -1.0
        end
    else
        return 0.0
    end
end

reward (generic function with 1 method)

In [43]:
# define the QuickPOMDP
m = QuickPOMDP(
#     states=GW_state,
    gen=gen,
    initialstate=initialstate,
    observation=observations,
    obstype=POMDPModelTools.BoolDistribution,
    actions=actions,
    reward=reward,
)


QuickPOMDP{UUID("c3e535ab-69b2-49cc-8eeb-c44e38afff51"),GW_state,Symbol,BoolDistribution,NamedTuple{(:isterminal, :actionindex, :initialstate, :reward, :gen, :actions, :discount, :obstype, :observation),Tuple{Bool,Dict{Symbol,Int64},typeof(initialstate),typeof(reward),typeof(gen),typeof(actions),Float64,DataType,typeof(observations)}}}((isterminal = false, actionindex = Dict(:left => 1,:right => 3,:stay => 2,:zoom_out => 6,:stay_zoom => 5,:track => 7,:zoom_in => 4), initialstate = initialstate, reward = reward, gen = gen, actions = actions, discount = 1.0, obstype = BoolDistribution, observation = observations))

In [24]:
using FIB # For the solver
using POMDPPolicies # For creating a random policy

In [25]:
# Define the FIB Solver with default params
solver = FIBSolver()

FIBSolver(100, 0.001, false)

In [44]:
# Solve the problem offline and obtain the FIB policy which is an AlphaVectorPolicy
fib_policy = solve(solver, m)

MissingQuickArgument: No definition of "states" for a QuickPOMDP (id=c3e535ab-69b2-49cc-8eeb-c44e38afff51).

Please specify it with a `states` keyword argument in the QuickPOMDP constructor.



In [28]:
# Create a policy that chooses actions at random
rand_policy = RandomPolicy(m);

In [45]:
# Create and run the rollout simulator
using POMDPSimulators
rollout_sim = RolloutSimulator(max_steps=10);

In [51]:
# fib_reward = simulate(rollout_sim, m, fib_policy);
rand_reward = simulate(rollout_sim, m, rand_policy);

UndefVarError: UndefVarError: simulate not defined

In [ ]:

@show fib_reward;
@show rand_reward;